# Compare Two Archived TensorFlow Models

Created by Mitas Ray on 2024-03-22.

This notebook is used to compare two TensorFlow models. The key ingredient is the `compare_models(...)` function which takes two arguments and compares the architectures and the weights for each layer. Each day a new model is trained and should be different from the previous day's model, and running this notebook allows one to check that.

To run the notebook, use `ficc_python/requirements_py310.txt`, and use `>>> pip install jupyter`.

In [14]:
import tensorflow as tf
import keras
import pandas as pd


# importing from parent directory: https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
sys.path.insert(0,'../')

from automated_training.auxiliary_functions import load_model_from_date, create_summary_of_results, create_input, fit_encoders
from automated_training.auxiliary_variables import CATEGORICAL_FEATURES

In [ ]:
model_03_08 = load_model_from_date('2024-03-08', folder='yield_spread_model', bucket='gs://automated_training')

In [ ]:
model_03_22 = load_model_from_date('2024-03-22', folder='yield_spread_model', bucket='gs://automated_training')

In [ ]:
def compare_models(model1, model2):
    '''Generated heavily by ChatGPT.'''
    # Get the number of layers in the model
    num_layers = len(model1.layers)

    # Initialize a list to store the absolute differences in weights for each layer
    weight_differences = []

    # Iterate through each layer and compare the weights
    for i in range(num_layers):
        # Get the weights of the current layer for both models
        weights_model1 = model1.layers[i].get_weights()
        weights_model2 = model2.layers[i].get_weights()
        
        # Calculate the absolute difference between the weights
        abs_diff = [tf.reduce_sum(tf.abs(w1 - w2)).numpy() for w1, w2 in zip(weights_model1, weights_model2)]
        
        # Store the absolute difference
        weight_differences.append(abs_diff)

    # Aggregate the comparison results
    average_abs_diff = sum(sum(diff) for diff in weight_differences) / sum(len(diff) for diff in weight_differences)

    # Print the average absolute difference
    print("Average absolute difference in weights:", average_abs_diff)

In [ ]:
compare_models(model_03_08, model_03_22)

In [ ]:
model_02_15 = load_model_from_date('2024-02-15', folder='yield_spread_model', bucket='gs://automated_training')

In [ ]:
compare_models(model_03_08, model_02_15)

# Model Performance Analysis: May 19 Model on June Data

This notebook evaluates the performance of our May 19th trained model on recent June trade data to assess model stability over time.

## Objective
Test whether models trained on May 19th data maintain acceptable performance (MAE) when applied to end-of-June data, potentially eliminating the need to retrain models during this time window.

## Process
1. Load the May 19th model from GCP bucket (`automated_training/`)
2. Load processed trade data from late June from GCP bucket (`automated_training/processed_data/`)
3. Perform inference using the May 19th model on June data
4. Calculate MAE (Mean Absolute Error) to quantify performance degradation
5. Compare results to determine if retraining is necessary for this period

## Expected Outcome
If MAE remains within acceptable thresholds, we can confidently use the May 19th model for this entire period, saving significant computational resources and training time while ensuring Ray Jay receives high-quality pricing data.

In [6]:
model_05_19 = keras.models.load_model("/Users/gil/git/ficc_python/notebooks/similar-trades-v2-model-2025-05-19")

In [7]:
df = pd.read_pickle("/Users/gil/git/ficc_python/notebooks/processed_data_yield_spread_with_similar_trades_v2.pkl")

In [12]:
len(df)

9838537

In [ ]:
test_data = df[df.trade_date == "2025-06-27"] 
test_data

,rtrs_control_number,cusip,yield,is_callable,refund_date,accrual_date,dated_date,next_sink_date,coupon,delivery_date,...,D_min_ago_ago,D_min_ago_qdiff,P_min_ago_ys,P_min_ago_ttypes,P_min_ago_ago,P_min_ago_qdiff,S_min_ago_ys,S_min_ago_ttypes,S_min_ago_ago,S_min_ago_qdiff
61643,2025062716781000,57582RJM9,506.0,True,NaT,2016-06-22,2016-06-22,NaT,3.00,2016-06-22,...,5.558251,3.699058,66.295648,PS,5.053482,3.699058,54.814849,SS,5.405406,4.740370
61644,2025062716781100,57582RJM9,507.0,True,NaT,2016-06-22,2016-06-22,NaT,3.00,2016-06-22,...,5.558251,3.699058,66.295648,PD,5.053482,3.699058,54.814849,SD,5.405406,4.740370
61645,2025062716471900,36665VBM2,321.5,True,NaT,2015-12-10,2015-12-10,2028-09-01,3.25,2015-12-10,...,5.422445,4.845104,65.901692,PS,5.789612,5.079185,22.446501,SS,5.422445,4.845104
61646,2025062716409500,42211HCN7,538.5,True,NaT,2020-06-23,2020-06-23,NaT,3.25,2020-06-23,...,4.321391,0.000000,177.822846,PS,4.321391,0.000000,283.527729,SS,8.206563,4.397957
61647,2025062716408600,13078YUT0,537.0,True,NaT,2023-05-17,2023-05-17,2037-09-02,5.25,2023-05-17,...,7.092170,5.146131,165.618526,PP,7.047366,0.000000,143.918526,SP,7.047366,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116367,2025062700031900,58731TBU3,303.2,False,NaT,2017-07-10,2017-07-10,NaT,4.00,2017-07-10,...,4.792476,4.000042,24.031838,PS,4.792476,4.000042,10.586002,SS,7.251781,0.000000
116368,2025062700031000,520351MF1,304.0,False,NaT,2023-12-14,2023-12-14,NaT,4.00,2023-12-14,...,4.853716,0.000000,10.099715,PD,4.853716,0.000000,-0.754957,SD,5.874510,4.000044
116369,2025062700030800,520351MF1,304.0,False,NaT,2023-12-14,2023-12-14,NaT,4.00,2023-12-14,...,4.853716,0.000000,10.099715,PS,4.853716,0.000000,-0.754957,SS,5.874510,4.000044
116370,2025062700013300,789177TJ1,452.0,True,NaT,2025-07-01,2025-07-01,NaT,0.00,2025-07-01,...,5.511263,5.439334,146.254881,PS,6.015806,5.176094,123.382793,SS,5.509864,5.439334


In [15]:
encoders, fmax = fit_encoders(df, CATEGORICAL_FEATURES, 'yield_spread_with_similar_trades')

In [16]:
x_test, y_test = create_input(test_data, encoders, "yield_spread_with_similar_trades")

BEGIN create_input
END create_input. Execution time: 0:00:00.292


In [17]:
create_summary_of_results(model_05_19, test_data, x_test, y_test)

 1/55 [..............................] - ETA: 1:12

2025-07-03 13:15:44.998097: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


55/55 [==============================] - 3s 30ms/step
|                                 |   Mean Absolute Error |   Trade Count |
|:--------------------------------|----------------------:|--------------:|
| Entire set                      |                25.196 |         54696 |
| Dealer-Dealer                   |                27.313 |         22203 |
| Bid Side / Dealer-Purchase      |                30.872 |         13399 |
| Offered Side / Dealer-Sell      |                18.753 |         19094 |
| AAA                             |                25.49  |          9082 |
| Investment Grade                |                23.94  |         42968 |
| Trade size >= 100k              |                23.317 |         11356 |
| Last trade <= 7 days            |                19.231 |         38179 |
| 7 days < Last trade <= 14 days  |                31.603 |          4587 |
| 14 days < Last trade <= 28 days |                47.485 |          4977 |
| 28 days < Last trade            

,Mean Absolute Error,Trade Count
Entire set,25.196,54696
Dealer-Dealer,27.313,22203
Bid Side / Dealer-Purchase,30.872,13399
Offered Side / Dealer-Sell,18.753,19094
AAA,25.490,9082
Investment Grade,23.940,42968
Trade size >= 100k,23.317,11356
Last trade <= 7 days,19.231,38179
7 days < Last trade <= 14 days,31.603,4587
14 days < Last trade <= 28 days,47.485,4977
